In [1]:
import numpy as np
import pandas as pd

In [2]:
# import reference table and daily file
ns = pd.read_csv('Thomasnet Customer Match Dataset.csv')
reference = pd.read_excel('Thomasnet Audience Reference.xlsx')
daily = pd.read_csv('82.csv')

In [3]:
ns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26675 entries, 0 to 26674
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Entity ID         26675 non-null  int64 
 1   Customer          26675 non-null  object
 2   Sales Rep         25404 non-null  object
 3   Seller Sales Rep  10443 non-null  object
 4   Customer ID       26675 non-null  object
 5   Date Created      26675 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [4]:
reference.columns

Index(['Visitor', 'Potential', 'New', 'Account ID', 'Sales Rep',
       'Seller Sales Rep'],
      dtype='object')

In [5]:
df1 = pd.merge(reference,ns,how='left',left_on='Account ID', right_on='Entity ID')

In [6]:
df1.head(10)

,Visitor,Potential,New,Account ID,Sales Rep_x,Seller Sales Rep_x,Entity ID,Customer,Sales Rep_y,Seller Sales Rep_y,Customer ID,Date Created
0,Esmond Natural USA Inc,No potential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eagle Labs,No potential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Synergy Labs,No potential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Big Storm Brewery,No Potential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Viscofan SA,Seller,NaN,37870,NaN,Katherine Salazar,37870.0,Naturin Viscofan GmbH,Unassigned Currently,Katherine Salazar,37870 Naturin Viscofan GmbH,4/25/2019 11:18 am
5,Nutrition Formulators Inc,Buyer,NaN,10124,Ana Aquino,NaN,10124.0,Nutrition Formulators,Ana Aquino,NaN,10124 Nutrition Formulators,6/18/2018 10:46 am
6,Myosci Technologies Inc,Both,Yes,44845,Elizabeth Suro,Katherine Salazar,44845.0,"Myosci Technologies, Inc./True Nutrition",Elizabeth Suro,Katherine Salazar,"44845 Myosci Technologies, Inc./True Nutrition",10/29/2020 12:27 pm
7,Texas Tech University,No Potential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Natural Organics Laboratories Inc,Buyer,NaN,11835,Tina Jun,NaN,11835.0,"Natural Organics,",Tina Jun,NaN,"11835 Natural Organics,",6/18/2018 10:45 am
9,Uniglo Manufacturing LLC,Buyer,NaN,40672,Ana Aquino,NaN,40672.0,UniGlo Manufacturing,Ana Aquino,NaN,40672 UniGlo Manufacturing,3/19/2020 2:11 pm


In [7]:
df1 = df1[['Visitor', 'Potential', 'New', 'Account ID', 'Sales Rep_y','Seller Sales Rep_y']]

In [8]:
df1.rename(columns={'Sales Rep_y':'Sales Rep','Seller Sales Rep_y':'Seller Sales Rep'},
                 inplace = True)

In [9]:
reference = df1

In [10]:
daily.index

RangeIndex(start=0, stop=1042, step=1)

In [11]:
daily.columns

Index(['Id', 'Time of Visit', 'Visitor', 'City', 'State/Province', 'Zip',
       'Country', 'Industry', 'Employees', 'Annual Sales', 'Pages/Sessions',
       'Session Duration', 'Search Term/Category', 'Thomasnet Registered User',
       'Your Website Activity', 'Thomasnet Activity', 'Target Audience',
       'Viewed High-Value Page', 'Thomasnet.com Submitted RFI',
       'Asset Download', 'Thomas Navigator Keyword/Part Number Search',
       'Thomas Navigator Advanced Searches/Attribute Column Sorts',
       'Thomas Navigator Page Saved to Favorites',
       'Thomas Navigator CAD Action', 'Thomas Navigator Compared Item',
       'Thomas Navigator Page Emailed', 'Thomas Navigator Printed Page',
       'Thomas Navigator Product Service View'],
      dtype='object')

In [12]:
daily = daily.drop_duplicates(subset = ['Visitor'],keep='first')

In [13]:
daily = daily.drop_duplicates().reset_index(drop=True)

In [14]:
# Clean daily file
# a. remove unknown values / gwi / EW
daily = daily[~daily.Visitor.str.contains("Unknown")]
daily = daily[~daily["Visitor"].isin(["Green Wave Ingredients","East -West Associates Inc"])]
# b. remove duplicates
daily.drop_duplicates("Visitor",keep='first',inplace=True)

In [15]:
b = len(daily)
b

243

In [16]:
# Join two tables based on the daily file
df = pd.merge(reference,daily,how='right')

# adjust column name and order
df['Visitor_2'] = df['Visitor']
df = df[['Visitor_2', 'Potential', 'New', 'Account ID', 'Sales Rep',
       'Seller Sales Rep', 'Id', 'Time of Visit', 'Visitor','City', 'State/Province',
       'Zip', 'Country', 'Industry', 'Employees', 'Annual Sales',
       'Pages/Sessions', 'Session Duration', 'Search Term/Category',
       'Thomasnet Registered User', 'Your Website Activity',
       'Thomasnet Activity', 'Target Audience', 'Viewed High-Value Page',
       'Thomasnet.com Submitted RFI', 'Asset Download',
       'Thomas Navigator Keyword/Part Number Search',
       'Thomas Navigator Advanced Searches/Attribute Column Sorts',
       'Thomas Navigator Page Saved to Favorites',
       'Thomas Navigator CAD Action', 'Thomas Navigator Compared Item',
       'Thomas Navigator Page Emailed', 'Thomas Navigator Printed Page',
       'Thomas Navigator Product Service View' ]]

# Replace nan with ''
df = df.replace(np.nan, '', regex=True) 


In [17]:
df.drop_duplicates("Visitor",keep='first',inplace=True)

In [18]:
a = len(df)
a

243

In [19]:
# c. Total daily visitors --> df_daily
df.to_csv("daily_total.csv",index=False)

In [20]:
# 2 tables:
# a. new visitors (not in reference table) --> df_new
# b. Existing visitors (with account#) --> df_report
# c. Total daily visitors --> df

In [21]:
# New visitors, need to qualify:
df_new = df[df['Potential'] == '']

# df_new[['New Customer']]
yes = ['Yes']*len(df_new)
df_new.insert(0,'New Customer',yes)
refer = ['']*len(df_new)
df_new.insert(7,'Referrer',refer)
web = ['']*len(df_new)
df_new.insert(8,'Website',web)

In [22]:
# df_new.drop_duplicates("Visitor",keep='first',inplace=True)
x=len(df_new)
x

46

In [23]:
# output to csv
df_new.to_csv("daily_new_visitors.csv",index=False)

In [24]:
# daily report visitors 
df_report = df[df['Account ID'] != '']
df_report = df_report[['Potential', 'New', 'Account ID', 'Sales Rep','Seller Sales Rep',
                       'Id', 'Time of Visit', 'Visitor','Country', 'Industry', 'Employees', 
                       'Annual Sales','Pages/Sessions', 'Session Duration', 'Search Term/Category']]

In [25]:
df_report.drop_duplicates("Visitor",keep='first',inplace=True)
df_report.sort_values(by=['Sales Rep'],ascending=True,inplace=True)
# df_report
c=len(df_report)
c

97

In [26]:
# output to csv
df_report.to_csv("daily_report.csv",index=False)